In [1]:
import numpy as np
import time
import random

In [2]:
def rastrigin(x):
    A = 10
    n = len(x)
    return A*n + sum(x[i]**2 - A*np.cos(2*np.pi*x[i]) for i in range(n))

In [3]:
def rosenbrock(x):
    a = 1
    b = 100
    return (a - x[0])**2 + b*(x[1] - x[0]**2)**2

In [4]:
def shaffer(x):
    return ( 0.5 + (np.sin((x[0]**2 +x[1]**2))**2-0.5)/(1.0 + 0.001*(x[0]**2+x[1]**2))**2)

In [5]:
class Particle:
    global_best_position = None
    global_best_value = None
    
    def __init__(self, bounds, obj_func, c_i,
                 c_p, c_g):
        self.lower_bounds = np.array([bound[0] for bound in bounds])
        self.upper_bounds = np.array([bound[1] for bound in bounds])
        
        self.position = np.random.uniform(self.lower_bounds, self.upper_bounds, len(bounds))
        self.velocity = np.random.uniform(self.lower_bounds - self.upper_bounds,
                                          self.upper_bounds - self.lower_bounds,
                                          len(bounds))

        self.personal_best_position = self.position.copy()
        
        self.obj_func = obj_func
        self.c_i = c_i
        self.c_p = c_p
        self.c_g = c_g
        
        self.value = obj_func(self.position)
        self.personal_best_value = obj_func(self.personal_best_position)

        if Particle.global_best_value is None or Particle.global_best_value > self.value:
            Particle.global_best_value = self.value
            Particle.global_best_position = self.position.copy()
        
    def update_position(self):
        self.position = np.clip(self.position + self.velocity,
                                self.lower_bounds,
                                self.upper_bounds)
        self.value = self.obj_func(self.position)
        if self.value < self.personal_best_value:
            self.personal_best_value = self.value
            self.personal_best_position = self.position.copy()
            if self.value < Particle.global_best_value:
                Particle.global_best_value = self.value
                Particle.global_best_position = self.position.copy()
    
    def update_velocity(self):
        r_p = np.random.random(self.velocity.shape)
        r_s = np.random.random(self.velocity.shape)
# mozda bi imalo smisla da normiramo vektore pre ovoga
# mozda i value ukljuciti pazljivo
        personal_direction=(self.personal_best_position - self.position)
        global_direction=(Particle.global_best_position - self.position)
        
        #if any([x>10**(-8) for x in personal_direction]):
         #   personal_direction/=np.linalg.norm(personal_direction)
        #if any([x!=0 for x in global_direction]):
         #   global_direction/=np.linalg.norm(global_direction)
        
        cognitive_velocity = r_p * self.c_p * (personal_direction)#*np.linalg.norm(self.velocity)
        social_velocity = r_s * self.c_g * (global_direction)#*np.linalg.norm(self.velocity)
        inertia = self.c_i * self.velocity
        self.velocity = np.clip(inertia + cognitive_velocity + social_velocity,2*self.lower_bounds,2*self.upper_bounds)
        

In [6]:

def pso(swarm_size, num_dimensions, c_i, c_p, c_g, allowed_time,func_to_minimize):
    bounds = [(-5.12, 5.12) for _ in range(num_dimensions)]
    
    swarm = [Particle(bounds=bounds,
                      obj_func=func_to_minimize,
                      c_i=c_i,
                      c_p=c_p,
                      c_g=c_g) for _ in range(swarm_size)]
    #print(Particle.global_best_value)
    start=time.time()
    while True:
        for particle in swarm:
            particle.update_velocity()
            particle.update_position()
        if time.time() - start>allowed_time:
            tmp=Particle.global_best_value
            Particle.global_best_position=None
            Particle.global_best_value=None
            return tmp

In [7]:
pso(swarm_size=50, num_dimensions=2, c_i=0.7, c_p=1.0, c_g=1.0, allowed_time=0.01,func_to_minimize=rosenbrock)

0.1906185174873264

In [8]:
times=[10**(-3),10**(-2),10**(-1),1]
num_trys=1000

In [9]:
 print("Function : Shaffer")
print("Swarm size: " + "50")
print("Num dimensions: " + "2")
print("c_i: "+"0.7")
print("c_p: " + "1.0")
print("c_g: " + "1.0")
for allowed_time in times:
    suma=0
    for _ in range(num_trys):
        suma+=pso(swarm_size=50, num_dimensions=2, c_i=0.7, c_p=1.0, c_g=1.0, allowed_time=allowed_time,func_to_minimize=shaffer)
    suma/=num_trys
   
    print("    Time : " + str(allowed_time))
    print("    Average mistake: "+ str(suma))
    print("\n")

Function : Shaffer
Swarm size: 50
Num dimensions: 2
c_i: 0.7
c_p: 1.0
c_g: 1.0
    Time : 0.001
    Average mistake: 0.005287713421096988


    Time : 0.01
    Average mistake: 4.9128414442081214e-05


    Time : 0.1
    Average mistake: 1.0414669127101206e-14


    Time : 1
    Average mistake: 0.0




In [10]:
rastrigin([1,2])

5.0